# Project 3: 'Image Analysis'
## First Year Project  
### ITU, Spring 2021

This notebook contains all the code developed to explore, wrangle and analyse the raw data sets for our project, 'Image Analysis'.

Contributors:  
- Andy Bao Nguyen (anbn)
- Florian Micliuc (flmi)
- Mattias Wohlert 
- Sofia Elena Terenziani (sote)

Created: 06-04-2021 

Last modified:

### Imports

In [21]:
import cv2
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
import timeit
from skimage import morphology
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

### Functions

### Loading the data

## Task 0 - Data checking and filtering

## Task 1 - Feature analysis

## Task 2 - Diagnosis prediction

## Task 3 - Open question

# WORK IN PROGRESS

In [22]:
#Example_ground_truth
#Features
ex_ground = pd.read_csv("../data/raw/example_ground_truth.csv")
features = pd.read_csv("../data/features/features.csv")
images = ("../data/raw/example_image/")
segmentations = ("../data/raw/example_segmentation/")

In [ ]:
# Create a DataFrame object
df_exground = pd.DataFrame(ex_ground, columns =['image_id', 'melanoma', 'seborrheic_keratosis'])

# Iterate over the index range from
# 0 to max number of columns in dataframe
for i in range(df_exground.shape[1]):

    print('Column Number : ', i)

    # Select column by index position using iloc[]
    columnSeriesObj = df_exground.iloc[:, i]
    print('Column Contents : ', columnSeriesObj.values)

In [ ]:
features

# Loading of data and scripts

In [23]:
%run ../Scripts/fyp2021p3_group00_functions.py

In [ ]:
im = plt.imread("../Data/Raw/example_image/ISIC_0014310.jpg")
print(im.shape)
plt.imshow(im)
plt.imshow(im[:,:,0], cmap='gray')

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    gray = 0.2989 * r * 0.5870 * g + 0.1140 * b
    return gray

grey = rgb2gray(im)
plt.imshow(grey,  cmap = "gray")

# Lesion is darker, and extract a mask of it

In [ ]:
plt.hist(grey)
# Will run for a minute or two

In [ ]:
# Our lesion is in the smaller clusters (The darker pixels), and the greater ones must be the pixels of the skin
im_lim = grey < 4500
plt.imshow(im_lim, cmap = "gray") # We see a little noice within our lesion

## Holy shit some messed op edge detections
The resized image testing here is somewhat crazy if you run it, you will get the image in gray, and an edge detection via Laplacian method.

In [27]:
test_image = cv2.imread("../Data/Raw/example_image/ISIC_0014310.jpg", cv2.IMREAD_GRAYSCALE)
imageS = cv2.resize(test_image, (960, 540))

laplacian_image = cv2.Laplacian(imageS, cv2.CV_64F)

cv2.imshow("Image", imageS)
cv2.imshow("Laplacian", laplacian_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Folder images multiple ways

In [ ]:
# This will get all the files in a folder in onlyfiles. And then it will read them all and store them in the array images.
onlyfiles = [f for f in os.listdir(images) if os.path.isfile(os.path.join(images, f))]
image_s = np.empty(len(onlyfiles), dtype = object)
for n in range(0, len(onlyfiles)):
    image_s[n] = cv2.imread(os.path.join(images, onlyfiles[n]))

In [ ]:
for i in image_s:
    print(i)

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
load_images_from_folder(images)